<a href="https://colab.research.google.com/github/Nekoiii/ML_Practices_colab/blob/main/train_own_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://tkstock.site/2023/05/19/googlecolab-stablediffusion-lora-finetuning/

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!git clone https://github.com/kohya-ss/sd-scripts
%cd sd-scripts

!pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 --extra-index-url https://download.pytorch.org/whl/cu117
!pip install --upgrade -r requirements.txt
!pip install -U --pre triton
!pip install xformers==0.0.16rc425

Cloning into 'sd-scripts'...
remote: Enumerating objects: 3003, done.
remote: Counting objects: 100% (1416/1416), done.
remote: Compressing objects: 100% (125/125), done.
remote: Total 3003 (delta 1323), reused 1314 (delta 1291), pack-reused 1587
Receiving objects: 100% (3003/3003), 3.30 MiB | 12.33 MiB/s, done.
Resolving deltas: 100% (2078/2078), done.
/content/sd-scripts
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu117
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.3/24.3 MB 69.6 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1+cu118
    Uninstalling torch-2.0.1+cu118:
      Successfully uninstalled torch-2.0.1+cu118
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.15.2+cu118
    Uninstalling torchvision-0.15.2+cu118:
      Successfully unin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 25.3 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 2.0.0
    Uninstalling triton-2.0.0:
      Successfully uninstalled triton-2.0.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 MB 36.0 MB/s eta 0:00:00


In [4]:
!accelerate config default --mixed_precision fp16

2023-06-08 22:59:36.129452: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-08 22:59:36.289244: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-08 22:59:36.328763: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-08 22:59:37.146729: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [5]:
# sd-scriptsに配下に訓練用画像をコピーする
%cd /content/sd-scripts
%cp -r /content/drive/MyDrive/ML_Practices/Lora_test_1/resized_sample_data_3 ./unitychan

/content/sd-scripts


In [6]:
!python finetune/make_captions.py --batch_size 8 ./unitychan
!python finetune/tag_images_by_wd14_tagger.py --batch_size 4 ./unitychan

2023-06-08 23:00:45.015406: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-08 23:00:45.172229: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-08 23:00:45.210275: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-08 23:00:46.001194: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [7]:
!python finetune/merge_captions_to_metadata.py ./unitychan meta_cap.json
!python finetune/merge_dd_tags_to_metadata.py ./unitychan --in_json meta_cap.json meta_cap_dd.json
!python finetune/clean_captions_and_tags.py meta_cap_dd.json meta_clean.json

2023-06-08 23:03:13.251203: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-08 23:03:13.408570: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-08 23:03:13.447361: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-08 23:03:14.236287: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [8]:
!python finetune/prepare_buckets_latents.py ./unitychan meta_clean.json meta_lat.json /content/drive/MyDrive/StableDifussion/models/anything-v4.5-pruned-fp16.ckpt --batch_size 4 --max_resolution 512,512 --mixed_precision no

2023-06-08 23:03:26.077401: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-08 23:03:26.232269: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-08 23:03:26.271615: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-08 23:03:27.077804: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [9]:
!accelerate launch  \
 --num_cpu_threads_per_process 1 train_network.py \
 --pretrained_model_name_or_path=/content/drive/MyDrive/StableDifussion/models/anything-v4.5-pruned-fp16.ckpt \
 --in_json meta_lat.json \
 --train_data_dir=./unitychan \
 --output_dir=output_models \
 --shuffle_caption \
 --train_batch_size=1 \
 --learning_rate=1e-4 \
 --max_train_steps=400 \
 --use_8bit_adam --xformers \
 --gradient_checkpointing \
 --mixed_precision=fp16 \
 --save_every_n_epochs=4 \
 --save_precision=fp16 \
 --save_model_as=safetensors  \
 --network_module=networks.lora

2023-06-08 23:04:03.041096: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-08 23:04:03.188382: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-08 23:04:03.226001: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-08 23:04:04.011938: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [10]:
!cp output_models/last.safetensors /content/drive/MyDrive/ML_Practices/Lora_test_2/output_models.safetensors

In [ ]:
!cat /content/sd-scripts/unitychan/*.txt